In [1]:
from tqdm import tqdm
import datetime as dt
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [ID Сайта], [Артикул поставщика], [Товарная группа], [Товарное направление], [Название товара], [Код склада], [Остаток], [Код товара]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'ЦБ', N'СИ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)

stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])
sklad1 = sklad1[sklad1['8'] != '8']

stock = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

stock = stock.drop(columns=['8'])

In [4]:
stock = stock[stock['ID Сайта'] != '']

In [5]:
ost = stock.groupby(by = 'ID Сайта', as_index = False)['Остаток'].sum()

In [6]:
stock.drop(columns=['Остаток', 'Код объекта', 'Код склада'], inplace=True)

In [7]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\ecomsales')
extension = 'xlsx'
all_sales = [i for i in glob.glob('*.{}'.format(extension))]

sale = pd.DataFrame()
for name in tqdm(all_sales):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

 94%|█████████▍| 17/18 [03:03<00:10, 10.80s/it]


PermissionError: [Errno 13] Permission denied: '~$3.2024_продажи еком.xlsx'

In [8]:
sale = sale.reset_index(drop=True)

sale = sale[(sale['state_order'] != 'returned') & (sale['state_order'] != 'canceled')]
sale = sale[~((sale['delivery_type'] == 'PICKUP_NOW') & (sale['payment_type'] == 'CASHIER') & (sale['type_ibox'] == 'CASHIER'))|
                ((sale['delivery_type'] == 'PICKUP') & (sale['payment_type'] == 'CASHIER') & (sale['type_ibox'] == 'CASHIER'))]

In [9]:
sale = sale[~sale['client_type'].str.contains('OZON')]

In [10]:
sale['result'] = sale['price_wo_manzana']
sale['result'].fillna(sale['price'], inplace=True)

In [11]:
def str_float(row):
    row = str(row)
    row = row.replace(',','.')
    return row

sale['price'] = sale['price'].apply(str_float)
sale['price'] = sale['price'].astype(float)
sale = sale[sale['price'] > 1].reset_index(drop=True)

In [12]:
sebes = sale.groupby(by = 'tov', as_index=False)['cost_price'].sum()

In [13]:
online = sale.groupby(by = 'tov', as_index=False)['result'].agg({'sum', 'count'})

In [14]:
online = online.merge(sebes, how='left', on='tov')

In [15]:
online.rename(columns={'tov': 'Код товара', 'sum': 'Сумма продаж', 'count': 'Количество продаж', 'cost_price': 'Себестоимость'}, inplace=True)

In [19]:
online

,Код товара,Сумма продаж,Количество продаж,Себестоимость
0,ТОВ0115302,33764.000000,1,15031.88
1,ТОВ0115418,4292.100000,1,1315.80
2,ТОВ0120217,8424.000000,1,3680.03
3,ТОВ0120218,8977.500000,1,4004.34
4,ТОВ0120219,16906.200000,1,4278.07
...,...,...,...,...
85408,ТОВ1334772,2970.000000,3,1337.20
85409,ТОВ1334773,3960.000000,4,1805.81
85410,ТОВ1334815,30821.315868,3,14865.07
85411,ТОВ1335284,1620.000000,3,783.72


In [17]:
sale = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Продажи по ТОВ СИ, ЦБ янв.23-июн.24.xlsx')

In [21]:
sale.rename(columns={'Номер': 'Код товара'}, inplace=True)

In [22]:
sale = sale.merge(online, how='left', on='Код товара')

In [23]:
sale

,Код товара,Количество-продажи,Сумма продаж с баллами,Себестоимость продаж,Сумма продаж,Количество продаж,Себестоимость
0,ТОВ0115302,5,198154.16,65596.85,33764.0,1.0,15031.88
1,ТОВ0115418,1,4292.10,1315.80,4292.1,1.0,1315.80
2,ТОВ0115554,1,66654.28,15108.45,NaN,NaN,NaN
3,ТОВ0119984,1,11000.00,2060.18,NaN,NaN,NaN
4,ТОВ0120217,1,8424.00,3680.03,8424.0,1.0,3680.03
...,...,...,...,...,...,...,...
104956,ТОВ1335466,2,10449.30,7512.00,NaN,NaN,NaN
104957,ТОВ1335467,4,23400.00,15412.00,NaN,NaN,NaN
104958,ТОВ1335468,1,6060.00,3986.00,NaN,NaN,NaN
104959,ТОВ1335469,2,12920.00,8498.00,NaN,NaN,NaN


In [26]:
sale.fillna(0, inplace=True)

In [27]:
sale['Сумма'] = sale['Сумма продаж'] + sale['Сумма продаж с баллами']
sale['Себес'] = sale['Себестоимость продаж'] + sale['Себестоимость']
sale['Кол-во'] = sale['Количество продаж'] + sale['Количество-продажи']

In [30]:
sale.drop(columns= ['Количество-продажи', 'Сумма продаж с баллами', 'Себестоимость продаж', 'Сумма продаж', 'Количество продаж', 'Себестоимость'],
        inplace = True)

In [33]:
stock = stock.merge(sale, how='left', on='Код товара')

In [36]:
ost

,ID Сайта,Остаток
0,1000030,41.0
1,1000181,58.0
2,1000188,675.0
3,1000336,24.0
4,1000457,1.0
...,...,...
47742,8380059,36.0
47743,8380065,13.0
47744,8380085,2.0
47745,8380087,3.0


In [37]:
stock = stock.merge(ost, how='left', on='ID Сайта')

In [39]:
stock.drop_duplicates(subset=['ID Сайта'], inplace=True)

In [41]:
cb = stock[stock['Товарное направление'] == 'ЦБ']
si = stock[stock['Товарное направление'] == 'СИ']

In [45]:
cb.sort_values(by=['Сумма'], ascending=False, inplace=True)
si.sort_values(by=['Сумма'], ascending=False, inplace=True)

In [49]:
cb = cb[:100]
si = si[:100]

In [50]:
result = pd.concat([cb, si], ignore_index=True)

In [52]:
images = result['ID Сайта']

In [54]:
images.to_excel('imagestop.xlsx', index=False)

0      1863703
1      1170467
2      8348333
3      1576139
4      1245677
        ...   
195    3618947
196    3701849
197    1160491
198    4866007
199    1950133
Name: ID Сайта, Length: 200, dtype: object